# Urban Mobility

[![Binder](http://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/anitagraser/ogd-at-lab/main?urlpath=lab/tree/notebooks/mobility.ipynb)

In [ ]:
import hvplot.pandas
from utils.dataaccess import get_uber_movement_gdf

## Uber Movement

Source: https://movement.uber.com/explore/vienna/travel-times

© 2021 Copyright Uber Technologies. Data made available under the [Creative Commons, Attribution Non-Commercial](https://creativecommons.org/licenses/by-nc/3.0/us/) license

In [ ]:
gdf = get_uber_movement_gdf()

In [ ]:
def plot_uber(source_id, attribute_col, month, *args, **kwargs):
    return ( gdf[(gdf.sourceid==source_id) & (gdf.month==month)].dropna(subset=[attribute_col]).hvplot(
            geo=True, tiles='OSM', c=attribute_col, title=f'{attribute_col.title()} - Month: 2020-{month}', *args, **kwargs) * 
        gdf[gdf.index==source_id].hvplot(geo=True).opts(active_tools=['wheel_zoom']))

In [ ]:
COL = 'mean_travel_time'
ID = 1
plot_uber(ID, COL, 1) + plot_uber(ID, COL, 2) + plot_uber(ID, COL, 3)

In [ ]:
jan = gdf[gdf.month==1].groupby('sourceid').count()
feb = gdf[gdf.month==2].groupby('sourceid').count()
mar = gdf[gdf.month==3].groupby('sourceid').count() 

In [ ]:
joined = feb.join(mar, lsuffix='_feb', rsuffix='_mar')
diff = joined[f'{COL}_mar'] - joined[f'{COL}_feb'] 
diff.hvplot.hist(title='Histogram of destination area counts (March 2020 - Feb 2020)')

In [ ]:
joined = jan.join(feb, lsuffix='_jan', rsuffix='_feb')
diff = joined[f'{COL}_feb'] - joined[f'{COL}_jan'] 
diff.hvplot.hist(title='Histogram of destination area counts (Feb 2020 - Jan 2020)')